# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like translations.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with language and words, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [ ]:
import time
import numpy as np
import tensorflow as tf

import sys
sys.path.insert(0, '../')
# import text_utils methods
import text_utils
import utils

from collections import Counter, namedtuple
import random

import pickle

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [ ]:
remote_path = 'http://mattmahoney.net/dc'
remote_filename = 'text8.zip'
dest_folder_path = 'data'

dst_filename = utils.download_zip_file(remote_path, remote_filename, dest_folder_path)

In [ ]:
with open(dst_filename) as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [ ]:
words = text_utils.preprocess(text)

In [ ]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

In [ ]:
vocab_to_int, int_to_vocab = text_utils.create_lookup_tables(words)
int_words = [vocab_to_int[w] for w in words]

## Save the vocabolary

In [ ]:
to_save = {'vocab_to_int': vocab_to_int, 'int_to_vocab': int_to_vocab}
with open('dictionary.cpkt', 'wb') as f:
    pickle.dump(to_save,f)

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. Check out my solution to see how I did it.

In [ ]:
def subsampling(words, threshold = 1e-5):
    words_count = Counter(words)
    total_count = len(words)
    freqs = {word: count/total_count for word, count in words_count.items()}
    p_drop = { word: 1 - np.sqrt(threshold/freqs[word]) for word in words_count}
    return [word for word in words if p_drop[word] < random.random()]
    

In [ ]:
train_words = subsampling(int_words)

In [ ]:
print("Total words: {}".format(len(train_words)))
print("Unique words: {}".format(len(set(train_words))))

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

In [ ]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index without duplicates.'''
    R = np.random.randint(1,window_size+1)
    start = np.max([0, idx - R])
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1: stop+1])
    return list(target_words)

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [ ]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = int(len(words) / batch_size)
    words = words[:n_batches * batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx: idx + batch_size]
        for i in range(len(batch)):
            batch_x = batch[i]
            batch_y = get_target(batch, i, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y    

## Building the graph

From Chris McCormick's blog, we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as one-hot encoded vectors. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. This weight matrix is usually called the embedding matrix or embedding look-up table. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

## Embedding


The embedding matrix has a size of the number of words by the number of neurons in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using one-hot encoded vectors for our inputs. When you do the matrix multiplication of the one-hot vector with the embedding matrix, you end up selecting only one row out of the entire matrix:

![one-hot matrix multiplication](assets/matrix_mult_w_one_hot.png)

You don't actually need to do the matrix multiplication, you just need to select the row in the embedding matrix that corresponds to the input word. Then, the embedding matrix becomes a lookup table, you're looking up a vector the size of the hidden layer that represents the input word.

<img src="assets/word2vec_weight_matrix_lookup_table.png" width=500>

## Negative sampling

For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).


## Build The Model

In [ ]:
def build_model(vocab_size, embed_dim, num_sampled=100):
    with tf.name_scope('Inputs'):
        inputs = tf.placeholder(tf.int32, [None], name='inputs')

    with tf.name_scope('Labels'):
        labels = tf.placeholder(tf.int32, [None, None], name='labels')

    with tf.name_scope('Embedding'):
        embeddings = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1.0, 1.0), name='embeddings')
        embed = tf.nn.embedding_lookup(embeddings, inputs, name='embed')

    with tf.name_scope('NegativeSampling'):
        softmax_w = tf.Variable(tf.truncated_normal((vocab_size, embed_dim), stddev=0.1), name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(vocab_size), name='softmax_b')
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b',softmax_b)
    #negative labels to sample
    with tf.name_scope('Cost'):
        loss = tf.nn.sampled_softmax_loss(softmax_w,softmax_b,labels,embed, num_sampled,vocab_size, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost',cost)

    with tf.name_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer(name='optimizer').minimize(cost)
        
    # merge all the summary in one node
    merged = tf.summary.merge_all()
    
    # Export the nodes
    export_nodes = ['inputs', 'labels', 'embeddings', 'embed', 
                    'softmax_w', 'softmax_b', 'cost', 'optimizer', 'merged']

    
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph        

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [ ]:
def validate(embeddings, sample_size = 16, sample_window = 100):
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    sample_examples = np.array(random.sample(range(sample_window), sample_size//2))
    sample_examples = np.append(sample_examples,
                               random.sample(range(1000,1000+sample_window), sample_size//2))
    
    sample_dataset = tf.constant(sample_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embedding = embeddings / norm
    sample_embedding = tf.nn.embedding_lookup(normalized_embedding, sample_dataset)
    similarity = tf.matmul(sample_embedding, tf.transpose(normalized_embedding))
    return similarity, sample_examples
    

## Training


Below is the code to train the network. Every 50 batches it reports the training loss. Every 100 batches, it'll print out the validation words.

In [ ]:
def train(epochs = 10, batch_size = 500, window_size = 10, print_every = 100, save_every = 1000):
    model = build_model(
        vocab_size = len(vocab_to_int),
        embed_dim = 300,
        num_sampled = 100
    )
    
    saver = tf.train.Saver(max_to_keep=50)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        file_writer = tf.summary.FileWriter('./logs/word2vec/', sess.graph)
        
        iteration = 1
        loss = 0
        for e in range(1, epochs+1):
            print('Epoch',e)
            batches = get_batches(train_words,batch_size, window_size)
            start = time.time()
            for x,y in batches:
#                 print('iteration', iteration)
                feed = {model.inputs: x, model.labels: np.array(y)[:, None]}
                summary, batch_loss, _ = sess.run([model.merged, model.cost, model.optimizer], feed_dict=feed)
                loss += batch_loss

                if iteration % print_every == 0: 
                    end = time.time()
                    print("Epoch {}/{}".format(e, epochs),
                          "Iteration: {}".format(iteration),
                          "Avg. Training loss: {:.4f}".format(loss/print_every),
                          "{:.4f} sec/batch".format((end-start)/print_every))
                    loss = 0
                    start = time.time()

                if iteration % save_every == 0:
                    sample_size = 16
                    # note that this is expensive (~20% slowdown if computed every 500 steps)
                    similarity, sample_examples = validate(model.embeddings, sample_size=sample_size)
                    sim = similarity.eval()
                    for i in range(sample_size):
                        sample_words = int_to_vocab[sample_examples[i]]
                        top_k = 5 # number of nearest neighbors
                        nearest = (-sim[i, :]).argsort()[1:top_k+1]
                        log = 'Nearest to %s:' % sample_words
                        for k in range(top_k):
                            close_words = int_to_vocab[nearest[k]]
                            log = '%s %s,' % (log, close_words)
                        print(log)
                    print('Saving checkpoint!')
                    saver.save(sess, './checkpoints/word2vec', global_step=iteration)

                iteration += 1
        
        #save the final model
        saver.save(sess, './checkpoints/word2vec', global_step=iteration)
    return model.embeddings

In [ ]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

In [ ]:
embeddings = train()